In [ ]:
import pennylane as qml
import numpy as np
from qiskit.quantum_info import DensityMatrix, partial_trace

# Number of qubits
n_qubits = 8

# Set the overlap threshold
mu = 0.5  # Desired maximum overlap

# Define the quantum devices
dev1 = qml.device('default.qubit', wires=n_qubits)
dev2 = qml.device('default.qubit', wires=n_qubits)

# Define the quantum circuit for Group 1 (Entangled States)
@qml.qnode(dev1)
def generate_entangled_state(params):
    qml.templates.AngleEmbedding(params, wires=range(n_qubits))
    # Apply CNOT gates to introduce entanglement
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    return qml.state()

# Define the quantum circuit for Group 2 (Product States)
@qml.qnode(dev2)
def generate_product_state(params):
    qml.templates.AngleEmbedding(params, wires=range(n_qubits))
    # No entangling gates applied here
    return qml.state()

# Function to generate states with overlap control
def generate_states_with_overlap_control(myu, num_states=10):
    group1 = []  # Entangled states
    group2 = []  # Product states

    while len(group1) < num_states or len(group2) < num_states:
        # Generate random parameters within distinct ranges
        params1 = np.random.uniform(0, np.pi/4, n_qubits)  # Narrow range for entangled states
        params2 = np.random.uniform(3 * np.pi / 4, np.pi, n_qubits)  # Different range for product states

        # Generate states
        psi1 = generate_entangled_state(params1)
        psi2 = generate_product_state(params2)

        # Compute overlap
        overlap = np.abs(np.vdot(psi1, psi2))**2
        print("overlap:",overlap)

        # Check if the overlap condition is satisfied
        if overlap < myu:
            if len(group1) < num_states:
                group1.append(psi1)
            if len(group2) < num_states:
                group2.append(psi2)

    return group1, group2

def calculate_Von_Neumann_entropy(state, qubits_to_trace_out):
    # Step 1: Calculate the full density matrix of the state
    density_matrix = DensityMatrix(state)

    # Step 2: Calculate the partial density matrix by tracing out specified qubits
    partial_density_matrix = partial_trace(density_matrix, qubits_to_trace_out)

    # Step 3: Get eigenvalues of the reduced density matrix
    eigenvalues = np.linalg.eigvalsh(partial_density_matrix.data)

    # Step 4: Filter out very small eigenvalues to avoid log(0)
    filtered_eigenvalues = [ev for ev in eigenvalues if ev > 1e-10]

    # Step 5: Calculate the Von Neumann entropy
    entropy = sum(-λ * np.log(λ) for λ in filtered_eigenvalues)

    # Step 6: Check if the state is entangled
    entangled = entropy > 1e-8

    return entropy, entangled


# Generate the states with the desired overlap
entangled_states, product_states = generate_states_with_overlap_control(myu=mu, num_states=10)

# # Calculate entropy and entanglement check for each state in the groups
# for idx, psi in enumerate(entangled_states + product_states):
#     entropy, entangled = calculate_Von_Neumann_entropy(psi, [2])  # Tracing out the 3rd qubit
#     print(f"State {idx + 1} - Von Neumann Entropy: {entropy}, Entangled: {entangled}")

# Display the generated states
print(f"Generated {len(entangled_states)} entangled states and {len(product_states)} product states.")
# print("Entangled States:", entangled_states)
# print("Product States:", product_states)

In [ ]:
import pennylane as qml
import numpy as np
import torch
from torch.optim import Adam
from sklearn.model_selection import train_test_split

# Number of qubits
n_qubits = 8

# Set the overlap threshold
mu = 1  # Desired maximum overlap

# Define the quantum device
dev = qml.device('default.qubit', wires=n_qubits)


@qml.qnode(dev, interface='torch')
def variational_classifier(params, inputs):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.StronglyEntanglingLayers(params, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0))

'''
# Cost function (Binary Cross Entropy Loss)
def cost(params, X, Y):
    predictions = [variational_classifier(params, torch.tensor(x, dtype=torch.float32)) for x in X]
    loss = torch.nn.functional.binary_cross_entropy_with_logits(
        torch.tensor(predictions, dtype=torch.float32, requires_grad=True),
        torch.tensor(Y, dtype=torch.float32)
    )
    return loss
'''
def cost(params, X, Y):
    predictions = torch.stack([variational_classifier(params, torch.tensor(x, dtype=torch.float32)) for x in X])
    loss = torch.nn.functional.binary_cross_entropy_with_logits(
        predictions,
        torch.tensor(Y, dtype=torch.float32)
    )
    return loss

# Modify generate_states_with_overlap_control to return parameters instead of states
def generate_states_with_overlap_control(myu, num_states=10):
    group1_params = []  # Parameters for entangled states
    group2_params = []  # Parameters for product states

    while len(group1_params) < num_states or len(group2_params) < num_states:
        # Generate random parameters within distinct ranges
        params1 = np.random.uniform(0, np.pi/4, n_qubits)  # Narrow range for entangled states
        params2 = np.random.uniform(3 * np.pi / 4, np.pi, n_qubits)  # Different range for product states

        # Generate states
        psi1 = generate_entangled_state(params1)
        psi2 = generate_product_state(params2)

        # Compute overlap
        overlap = np.abs(np.vdot(psi1, psi2))**2

        # Check if the overlap condition is satisfied
        if overlap < myu:
            if len(group1_params) < num_states:
                group1_params.append(params1)
            if len(group2_params) < num_states:
                group2_params.append(params2)

    return group1_params, group2_params

# Generate dataset (entangled and product states based on parameters)
entangled_params, product_params = generate_states_with_overlap_control(myu=mu, num_states=100)

# Prepare input dataset and labels (1 for entangled, 0 for product states)
X = np.array(entangled_params + product_params)
Y = np.array([1] * len(entangled_params) + [0] * len(product_params))

# Split the dataset into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
# Print shapes
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)
print()
# Initialize parameters for StronglyEntanglingLayers
params_shape = qml.templates.StronglyEntanglingLayers.shape(n_layers=2, n_wires=n_qubits)
#params = torch.randn(params_shape, requires_grad=True)  # Make sure params require gradients
params = torch.randn(params_shape, requires_grad=True)   # Scale down the initial parameters

# Set optimizer
optimizer = Adam([params], lr=0.001)  # Try a smaller learning rate
n_epochs = 1000

# Training loop
for epoch in range(n_epochs):
    optimizer.zero_grad()
    #torch.nn.utils.clip_grad_norm_(params, max_norm=1.0)
    loss = cost(params, X_train, Y_train)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item()}")

# Test the model
predictions = [variational_classifier(params, torch.tensor(x, dtype=torch.float32)).detach().numpy() for x in X_test]
accuracy = np.mean([(p >= 0 and y == 1) or (p < 0 and y == 0) for p, y in zip(predictions, Y_test)])
print(f"Test Accuracy: {accuracy * 100:.2f}%")

import pennylane as qml
import numpy as np
import torch
from torch.optim import Adam
from sklearn.model_selection import train_test_split

# Number of qubits
n_qubits = 8

# Set the overlap threshold
mu = 1  # Desired maximum overlap

# Define the quantum device
dev = qml.device('default.qubit', wires=n_qubits)


@qml.qnode(dev, interface='torch')
def variational_classifier(params, inputs):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.StronglyEntanglingLayers(params, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0))

'''
# Cost function (Binary Cross Entropy Loss)
def cost(params, X, Y):
    predictions = [variational_classifier(params, torch.tensor(x, dtype=torch.float32)) for x in X]
    loss = torch.nn.functional.binary_cross_entropy_with_logits(
        torch.tensor(predictions, dtype=torch.float32, requires_grad=True),
        torch.tensor(Y, dtype=torch.float32)
    )
    return loss
'''
def cost(params, X, Y):
    predictions = torch.stack([variational_classifier(params, torch.tensor(x, dtype=torch.float32)) for x in X])
    loss = torch.nn.functional.binary_cross_entropy_with_logits(
        predictions,
        torch.tensor(Y, dtype=torch.float32)
    )
    return loss

# Modify generate_states_with_overlap_control to return parameters instead of states
def generate_states_with_overlap_control(myu, num_states=10):
    group1_params = []  # Parameters for entangled states
    group2_params = []  # Parameters for product states

    while len(group1_params) < num_states or len(group2_params) < num_states:
        # Generate random parameters within distinct ranges
        params1 = np.random.uniform(0, np.pi/4, n_qubits)  # Narrow range for entangled states
        params2 = np.random.uniform(3 * np.pi / 4, np.pi, n_qubits)  # Different range for product states

        # Generate states
        psi1 = generate_entangled_state(params1)
        psi2 = generate_product_state(params2)

        # Compute overlap
        overlap = np.abs(np.vdot(psi1, psi2))**2

        # Check if the overlap condition is satisfied
        if overlap < myu:
            if len(group1_params) < num_states:
                group1_params.append(params1)
            if len(group2_params) < num_states:
                group2_params.append(params2)

    return group1_params, group2_params

# Generate dataset (entangled and product states based on parameters)
entangled_params, product_params = generate_states_with_overlap_control(myu=mu, num_states=100)

# Prepare input dataset and labels (1 for entangled, 0 for product states)
X = np.array(entangled_params + product_params)
Y = np.array([1] * len(entangled_params) + [0] * len(product_params))

# Split the dataset into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
# Print shapes
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)
print()
# Initialize parameters for StronglyEntanglingLayers
params_shape = qml.templates.StronglyEntanglingLayers.shape(n_layers=2, n_wires=n_qubits)
#params = torch.randn(params_shape, requires_grad=True)  # Make sure params require gradients
params = torch.randn(params_shape, requires_grad=True)   # Scale down the initial parameters

# Set optimizer
optimizer = Adam([params], lr=0.001)  # Try a smaller learning rate
n_epochs = 1000

# Training loop
for epoch in range(n_epochs):
    optimizer.zero_grad()
    #torch.nn.utils.clip_grad_norm_(params, max_norm=1.0)
    loss = cost(params, X_train, Y_train)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item()}")

# Test the model
predictions = [variational_classifier(params, torch.tensor(x, dtype=torch.float32)).detach().numpy() for x in X_test]
accuracy = np.mean([(p >= 0 and y == 1) or (p < 0 and y == 0) for p, y in zip(predictions, Y_test)])
print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [ ]:
import pennylane as qml
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

# Number of qubits
num_qubits = 8

# Set the overlap thresholds
mu_min = 0.3  # Minimum overlap
mu_max = 0.5  # Maximum overlap

# Define the quantum devices
dev1 = qml.device('default.qubit', wires=num_qubits)
dev2 = qml.device('default.qubit', wires=num_qubits)

# Define the quantum circuit for Group 1 (Entangled States)
@qml.qnode(dev1)
def generate_entangled_state(params):
    qml.templates.AngleEmbedding(params, wires=range(num_qubits))
    # Apply CNOT gates to introduce entanglement
    for i in range(num_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    return qml.state()

# Define the quantum circuit for Group 2 (Product States)
@qml.qnode(dev2)
def generate_product_state(params):
    qml.templates.AngleEmbedding(params, wires=range(num_qubits))
    # No entangling gates applied here
    return qml.state()

# Function to generate states with overlap control
def generate_states_with_overlap_control(mu_min, mu_max, num_states=10):
    group1 = []  # Entangled states
    group2 = []  # Product states

    while len(group1) < num_states or len(group2) < num_states:
        # Generate random parameters within distinct ranges
        params1 = np.random.uniform(0, np.pi/4, num_qubits)  # Narrow range for entangled states
        params2 = np.random.uniform(3 * np.pi / 4, np.pi, num_qubits)  # Different range for product states

        # Generate states
        psi1 = generate_entangled_state(params1)
        psi2 = generate_product_state(params2)

        # Compute overlap
        overlap = np.abs(np.vdot(psi1, psi2))**2
        print("overlap: ",overlap)
        # Check if the overlap is within the desired range
        if mu_min < overlap < mu_max:
            if len(group1) < num_states:
                group1.append(psi1)
            if len(group2) < num_states:
                group2.append(psi2)

    return group1, group2


In [ ]:
from torch.optim import ASGD
optimizer = ASGD([params], lr=0.001)  # Try a smaller learning rate
import matplotlib.pyplot as plt

# Initialize lists to store loss and accuracy values
loss_values = []
accuracy_values = []
accuracy_epochs = []
n_epochs = 100

# Training loop with logging
for epoch in range(n_epochs):
    optimizer.zero_grad()
    loss = cost(params, X_train, Y_train)
    loss.backward()
    optimizer.step()

    # Log the loss value
    loss_values.append(loss.item())

    if epoch % 5 == 0:
        # Calculate accuracy on the test set
        predictions = [variational_classifier(params, torch.tensor(x, dtype=torch.float32)).detach().numpy() for x in X_test]
        accuracy = np.mean([(p >= 0 and y == 1) or (p < 0 and y == 0) for p, y in zip(predictions, Y_test)])
        accuracy_values.append(accuracy)
        accuracy_epochs.append(epoch)
        print(f"Epoch {epoch}: Loss = {loss.item()}, Test Accuracy = {accuracy * 100:.2f}%")


# Test the model
predictions = [variational_classifier(params, torch.tensor(x, dtype=torch.float32)).detach().numpy() for x in X_test]
accuracy = np.mean([(p >= 0 and y == 1) or (p < 0 and y == 0) for p, y in zip(predictions, Y_test)])
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Plot the loss over epochs
plt.figure(figsize=(12, 6))
plt.plot(loss_values, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.legend()
plt.grid(True)
plt.show()

# Plot the accuracy over epochs (every 10 epochs)
plt.figure(figsize=(12, 6))
plt.plot(accuracy_epochs, accuracy_values, label='Test Accuracy', marker='o')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Test Accuracy over Epochs')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from torch.optim import SGD
optimizer = SGD([params], lr=0.001)  # Try a smaller learning rate
import matplotlib.pyplot as plt

# Initialize lists to store loss and accuracy values
loss_values = []
accuracy_values = []
accuracy_epochs = []
n_epochs = 100

# Training loop with logging
for epoch in range(n_epochs):
    optimizer.zero_grad()
    loss = cost(params, X_train, Y_train)
    loss.backward()
    optimizer.step()

    # Log the loss value
    loss_values.append(loss.item())

    if epoch % 5 == 0:
        # Calculate accuracy on the test set
        predictions = [variational_classifier(params, torch.tensor(x, dtype=torch.float32)).detach().numpy() for x in X_test]
        accuracy = np.mean([(p >= 0 and y == 1) or (p < 0 and y == 0) for p, y in zip(predictions, Y_test)])
        accuracy_values.append(accuracy)
        accuracy_epochs.append(epoch)
        print(f"Epoch {epoch}: Loss = {loss.item()}, Test Accuracy = {accuracy * 100:.2f}%")


# Test the model
predictions = [variational_classifier(params, torch.tensor(x, dtype=torch.float32)).detach().numpy() for x in X_test]
accuracy = np.mean([(p >= 0 and y == 1) or (p < 0 and y == 0) for p, y in zip(predictions, Y_test)])
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Plot the loss over epochs
plt.figure(figsize=(12, 6))
plt.plot(loss_values, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.legend()
plt.grid(True)
plt.show()

# Plot the accuracy over epochs (every 10 epochs)
plt.figure(figsize=(12, 6))
plt.plot(accuracy_epochs, accuracy_values, label='Test Accuracy', marker='o')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Test Accuracy over Epochs')
plt.legend()
plt.grid(True)
plt.show()